In [1]:
import sys
sys.path.append("../")

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import os
import matplotlib.pyplot as plt

from datasets.llff import LLFFDataset
from datasets.llff import *

In [4]:
root_dir = "/scratch/saksham/data/freiburg_small/"

In [5]:
from datasets.colmap_utils import \
    read_cameras_binary, read_images_binary, read_points3d_binary

In [6]:
img_wh = (80,60)

camdata = read_cameras_binary(os.path.join(root_dir, 'sparse/0/cameras.bin'))
H = camdata[1].height
W = camdata[1].width
focal = camdata[1].params[0] * img_wh[0]/W

In [7]:
imdata = read_images_binary(os.path.join(root_dir, 'sparse/0/images.bin'))

In [8]:
perm = np.argsort([imdata[k].name for k in imdata])

In [9]:
image_paths = [os.path.join(root_dir, 'images', name)
                            for name in sorted([imdata[k].name for k in imdata])]

In [10]:
image_paths = [os.path.join(root_dir, 'images', name)
                            for name in sorted([imdata[k].name for k in imdata])]

In [11]:
w2c_mats = []
bottom = np.array([0, 0, 0, 1.]).reshape(1, 4)
for k in imdata:
    im = imdata[k]
    R = im.qvec2rotmat()
    t = im.tvec.reshape(3, 1)
    w2c_mats += [np.concatenate([np.concatenate([R, t], 1), bottom], 0)]
w2c_mats = np.stack(w2c_mats, 0)
poses = np.linalg.inv(w2c_mats)[:, :3] # (N_images, 3, 4) cam2world matrices

In [12]:
bounds = np.zeros((len(poses), 2)) # (N_images, 2)
pts3d = read_points3d_binary(os.path.join(root_dir, 'sparse/0/points3D.bin'))

In [13]:
pts_world = np.zeros((1, 3, len(pts3d))) # (1, 3, N_points)
visibilities = np.zeros((len(poses), len(pts3d))) # (N_images, N_points)
for i, k in enumerate(pts3d):
    pts_world[0, :, i] = pts3d[k].xyz
    for j in pts3d[k].image_ids:
        visibilities[j-1, i] = 1
        

In [14]:
depths = ((pts_world-poses[..., 3:4])*poses[..., 2:3]).sum(1) # (N_images, N_points)
for i in range(len(poses)):
    visibility_i = visibilities[i]
    zs = depths[i][visibility_i==1]
    bounds[i] = [np.percentile(zs, 0.1), np.percentile(zs, 99.9)]
# permute the matrices to increasing order
poses = poses[perm]
bounds = bounds[perm]

In [15]:
poses.shape

(101, 3, 4)

In [16]:
poses = np.concatenate([poses[..., 0:1], -poses[..., 1:3], poses[..., 3:4]], -1)

In [17]:
poses = poses[:30]
image_paths = image_paths[:30]
bounds = bounds[:30]

In [18]:
poses, _ = center_poses(poses)

In [19]:
distances_from_center = np.linalg.norm(poses[..., 3], axis=1)

In [20]:
val_idx = np.argmin(distances_from_center) # choose val image as the closest to
                                                   # center image

In [21]:
near_original = bounds.min()

In [22]:
near_original

2.0559904843130434

In [23]:
directions = get_ray_directions(img_wh[1], img_wh[0], focal) # (H, W, 3)

In [24]:
index = 5

c2w = torch.FloatTensor(poses[index])

In [25]:
rays_o, rays_d = get_rays(directions, c2w) 

In [26]:
H = img_wh[1]
W = img_wh[0]
near = 1

In [27]:
rays_o

tensor([[0.8979, 0.3870, 0.0262],
        [0.8979, 0.3870, 0.0262],
        [0.8979, 0.3870, 0.0262],
        ...,
        [0.8979, 0.3870, 0.0262],
        [0.8979, 0.3870, 0.0262],
        [0.8979, 0.3870, 0.0262]])

In [28]:
rays_o[...,2]

tensor([0.0262, 0.0262, 0.0262,  ..., 0.0262, 0.0262, 0.0262])

In [29]:
t = -(near + rays_o[...,2]) / rays_d[...,2]

In [30]:
t[...,None]

tensor([[1.3846],
        [1.3749],
        [1.3655],
        ...,
        [1.1647],
        [1.1695],
        [1.1744]])

In [31]:
t

tensor([1.3846, 1.3749, 1.3655,  ..., 1.1647, 1.1695, 1.1744])

In [32]:
t = -(near + rays_o[...,2]) / rays_d[...,2]
rays_o = rays_o + t[...,None] * rays_d

# Store some intermediate homogeneous results
ox_oz = rays_o[...,0] / rays_o[...,2]
oy_oz = rays_o[...,1] / rays_o[...,2]

# Projection
o0 = -1./(W/(2.*focal)) * ox_oz
o1 = -1./(H/(2.*focal)) * oy_oz
o2 = 1. + 2. * near / rays_o[...,2]

d0 = -1./(W/(2.*focal)) * (rays_d[...,0]/rays_d[...,2] - ox_oz)
d1 = -1./(H/(2.*focal)) * (rays_d[...,1]/rays_d[...,2] - oy_oz)
d2 = 1 - o2

rays_o = torch.stack([o0, o1, o2], -1) # (B, 3)
rays_d = torch.stack([d0, d1, d2], -1) # (B, 3)

In [33]:
rays_o

tensor([[ 0.1366,  1.8980, -1.0000],
        [ 0.1663,  1.9014, -1.0000],
        [ 0.1959,  1.9048, -1.0000],
        ...,
        [ 2.3440,  0.2095, -1.0000],
        [ 2.3666,  0.2146, -1.0000],
        [ 2.3892,  0.2198, -1.0000]])

In [34]:
rays_d

tensor([[-1.5084, -0.9135,  2.0000],
        [-1.5092, -0.9136,  2.0000],
        [-1.5100, -0.9136,  2.0000],
        ...,
        [-1.5649, -0.8703,  2.0000],
        [-1.5655, -0.8704,  2.0000],
        [-1.5660, -0.8706,  2.0000]])